In [ ]:
%matplotlib inline

import os

# let's not pollute this blog post with warnings
from warnings import filterwarnings
filterwarnings('ignore')

import keras
import numpy as np
import pandas as pd
import skvideo.io as skv
from tqdm import tqdm

In [ ]:
# load the data
run = '-mlp-reg-micro'
labelpath = os.path.join('train_labels.csv')
train_labels = pd.read_csv(labelpath, index_col='filename')

In [ ]:
train_labels.head()

In [ ]:
train_labels.info()

In [ ]:
train_labels.sum(axis=0).sort_values(ascending=False)

In [ ]:
(train_labels.sum(axis=1) > 1).sum()

In [ ]:
from primatrix_dataset_utils import Dataset

In [ ]:
datapath = os.path.join('.')
whichset = run.split('-')[-1]
redframes = whichset == 'nano'
data = Dataset(datapath=datapath, 
               dataset_type=whichset,
               reduce_frames=redframes, 
               batch_size=32, 
               test=False)

In [ ]:
data.num_classes

In [ ]:
data.num_frames

In [ ]:
data.batch_size

In [ ]:
data.num_samples

In [ ]:
data.y_train.shape[0]

In [ ]:
data.y_val.shape[0]

In [ ]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

print(get_available_gpus())

In [ ]:
from keras.models import Sequential
from keras.layers import TimeDistributed, Conv2D, MaxPooling2D, Flatten, Dropout, Dense, BatchNormalization
from keras.layers.recurrent import LSTM
from keras.callbacks import ModelCheckpoint

In [ ]:
# instantiate model
model = Sequential()

# add three time-distributed convolutional layers for feature extraction
model.add(BatchNormalization(input_shape=(data.num_frames, data.width, data.height, 3)))
model.add(TimeDistributed(Conv2D(64, (5, 5), activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Conv2D(128, (3,3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Conv2D(256, (3,3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Conv2D(512, (3,3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Conv2D(1024, (2,2), activation='relu')))


# extract features and dropout 
model.add(Flatten())
#model.add(Dropout(0.2))

# input to LSTM
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

# classifier with sigmoid activation for multilabel
model.add(Dense(data.num_classes, activation='sigmoid'))

adam = keras.optimizers.Adam(lr=0.001, decay=0.0)

# compile the model with binary_crossentropy loss for multilabel
model.compile(optimizer=adam, loss='binary_crossentropy')

model_name = 'model' + run + '.h5'
checkpoint = ModelCheckpoint(model_name, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]


# look at the params before training
model.summary()

In [ ]:
model.fit_generator(
    data.batches(), 
    steps_per_epoch=int(data.num_batches/20),                  # data.num_batches to train on full set 
    epochs=6000, 
    validation_data=data.val_batches(), 
    validation_steps=int(data.num_val_batches/20),                  # data.num_val_batches to validate on full set
    callbacks=callbacks_list
)

In [ ]:
# load model
from keras.models import load_model

trained_model = load_model(model_name)

# generate predictions
for batch_num in tqdm(range(data.num_test_batches), total=data.num_test_batches):

    # make predictions on batch
    results = trained_model.predict_proba(next(data.test_batches()), 
                                          batch_size=data.batch_size, 
                                          verbose=0)

    # update submission format dataframe stored in dataset object
    data.update_predictions(results)          
    
data.predictions.to_csv(os.path.join(data.datapath, 'predictions' + run + '.csv'))

In [ ]:
!head -n 5 ./predictions.csv